In [1]:
import copy
from IPython.display import display, Image
import logging
import numpy
import os
import pandas
import pdb
import plotly
import pprint
import pyarrow
import pyarrow.parquet as pq
import six
import sys
import tensorflow as tf
import time

import CS230

plotly.offline.init_notebook_mode(connected=True)

In [2]:
logger = CS230.common.get_logger(level=logging.DEBUG)

In [3]:
file_paths = CS230.data.get_all_file_paths()
logger.debug(file_paths)

01-48-01 | INFO | num files: 22
01-48-01 | DEBUG | ['/home/dave/code/github/CS230_project/data/2013_Monterey_Motorsports_Reunion/20130810_01_01_01_grandsport.parquet', '/home/dave/code/github/CS230_project/data/2013_Monterey_Motorsports_Reunion/20130810_02_01_01_grandsport.parquet', '/home/dave/code/github/CS230_project/data/2013_Monterey_Motorsports_Reunion/20130811_01_01_01_grandsport.parquet', '/home/dave/code/github/CS230_project/data/2013_Monterey_Motorsports_Reunion/20130811_02_01_01_grandsport.parquet', '/home/dave/code/github/CS230_project/data/2013_Monterey_Motorsports_Reunion/20130815_01_01_02_grandsport.parquet', '/home/dave/code/github/CS230_project/data/2013_Monterey_Motorsports_Reunion/20130816_01_01_02_grandsport.parquet', '/home/dave/code/github/CS230_project/data/2013_Monterey_Motorsports_Reunion/20130817_01_01_02_grandsport.parquet', '/home/dave/code/github/CS230_project/data/2013_Monterey_Motorsports_Reunion/20130817_02_01_02_grandsport.parquet', '/home/dave/code/git

# load and process data

In [4]:
file_path = file_paths[0]
df = CS230.data.load(file_path)
df = CS230.data.stride_rows(df, stride=10)
df = CS230.data.add_derivatives(df, stride=1)
df = CS230.data.clean_discontinuities(df, stride=1)

01-48-01 | DEBUG | # deriv_yawAngle fixed: 42
01-48-01 | DEBUG | GPS NaN at 6792 :   0.50,  0.00,  5.29, -0.05, 48.93, -1.23,  3.40 -> -0.14, 1.02, 0.00, 0.00, 0.00, 0.00, 0.00


# plot data

In [7]:
fig, image_path = CS230.data.plot(df, file_path, columns=CS230.data.COLUMNS_ALL, plot=True, write=False,
                                     start=128300, stop=128400, step=1)

fig, image_path = CS230.data.plot(df, file_path, columns=CS230.data.COLUMNS_DERIV, plot=True, write=False,
                                     start=128300, stop=128400, step=1)

01-48-42 | DEBUG | skipping column: distance
01-48-42 | DEBUG | skipping column: time


# split data into training / dev / test sets

In [9]:
train_percent = 0.9
dev_percent = 0.05
test_percent = 0.05

df_train, df_dev, df_test = CS230.data.get_data_sets(df, train_percent, dev_percent, test_percent, 
                                                     CS230.data.COLUMNS_HUMAN_INPUT, CS230.data.COLUMNS_DERIV)

In [10]:
df_train.head()

,brake,throttle,handwheelAngle,deriv_axCG,deriv_ayCG,deriv_pitchAngle,deriv_pitchRate,deriv_rollAngle,deriv_rollRate,deriv_vxCG,deriv_vyCG,deriv_wheelAccelFL,deriv_wheelAccelFR,deriv_wheelAccelRL,deriv_wheelAccelRR,deriv_yawAngle,deriv_yawRate
0,0.0,2.0,352.7,0.42,0.67,-0.03,-0.11,0.06,1.90,-0.01,0.00,2.26,-2.26,-0.69,-0.23,0.14,0.14
1,0.0,0.8,-33.1,-0.02,0.01,-0.01,0.32,0.00,-0.10,0.00,0.00,0.00,-0.20,0.20,0.38,0.00,0.25
2,0.0,36.8,-61.3,-0.97,3.61,0.02,1.10,0.00,7.27,0.04,0.05,5.30,-0.49,6.48,-1.22,0.13,-1.20
3,0.0,82.1,2.0,2.86,-1.85,0.06,6.72,0.06,4.80,0.05,-0.03,3.34,-3.63,-11.48,-19.47,0.03,0.43
4,0.0,0.9,-5.6,0.01,-0.05,0.00,0.09,0.01,-0.27,0.00,-0.00,-0.09,0.40,0.19,0.27,0.01,-0.21


# train model

In [11]:
model = tf.keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(tf.keras.layers.Dense(len(CS230.data.COLUMNS_HUMAN_INPUT), activation='relu'))
# Add another:
model.add(tf.keras.layers.Dense(len(CS230.data.COLUMNS_DERIV), activation='relu'))

In [12]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
x = tf.cast(df_train[CS230.data.COLUMNS_HUMAN_INPUT].values, tf.float32)
y = tf.cast(df_train[CS230.data.COLUMNS_DERIV].values, tf.float32)

parameters = model.fit(x, y, epochs=5, steps_per_epoch=1)

Epoch 1/5
1/1 [==============================] - 0s 33ms/step - loss: nan - acc: 0.0119
Epoch 2/5
1/1 [==============================] - 0s 38ms/step - loss: nan - acc: 0.0119
Epoch 3/5
1/1 [==============================] - 0s 37ms/step - loss: nan - acc: 0.0119
Epoch 4/5
1/1 [==============================] - 0s 37ms/step - loss: nan - acc: 0.0119
Epoch 5/5
1/1 [==============================] - 0s 37ms/step - loss: nan - acc: 0.0119
